# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [74]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats
from statistics import mean

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [75]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN

['pk_a1b61b6073bf49b2b23b8d196150e6cb', 'Tpk_350573cf340f451db19d8484c64d009b']

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [76]:
symbol ='AAPL'
base_url = "https://sandbox.iexapis.com/stable"
api_url = f"/stock/market/batch?symbols={symbol}&types=quote&token={IEX_CLOUD_API_TOKEN[-1]}"

data = requests.get(base_url+api_url).json()
data

{'AAPL': {'quote': {'avgTotalVolume': 108971493,
   'calculationPrice': 'close',
   'change': 4.95,
   'changePercent': 0.02899,
   'close': 187.68,
   'closeSource': 'ficoiafl',
   'closeTime': 1669193301646,
   'companyName': 'Apple Inc',
   'currency': 'USD',
   'delayedPrice': 180.5,
   'delayedPriceTime': 1705074772545,
   'extendedChange': 0.4,
   'extendedChangePercent': 0.0022,
   'extendedPrice': 187.45,
   'extendedPriceTime': 1683579397849,
   'high': 187.68,
   'highSource': '1yenedpcem5 ur  eltadii',
   'highTime': 1675789656570,
   'iexAskPrice': None,
   'iexAskSize': None,
   'iexBidPrice': None,
   'iexBidSize': None,
   'iexClose': 184.976,
   'iexCloseTime': 1691520306388,
   'iexLastUpdated': None,
   'iexMarketPercent': None,
   'iexOpen': 181.03,
   'iexOpenTime': 1653856074930,
   'iexRealtimePrice': None,
   'iexRealtimeSize': None,
   'iexVolume': None,
   'lastTradeTime': 1692786860860,
   'latestPrice': 179.76,
   'latestSource': 'Close',
   'latestTime': 'De

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [77]:
data[symbol]['quote']['latestPrice']
data[symbol]['quote']['peRatio']

16.22

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [78]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [79]:
final_dataframe = pd.DataFrame(columns=my_columns)


for symbol_string in symbol_strings:
    api_url = f"/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN[-1]}"
    data = requests.get(base_url+api_url).json()
    data
    for symbol in symbol_string.split(","):
        final_dataframe=final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                "N/A"
            ],index=my_columns
            ),ignore_index=True
        )
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,161.44,50.47,N/A
1,AAL,18.04,-3.65,N/A
2,AAP,251.64,24.7,N/A
3,AAPL,182.41,16.33,N/A
4,ABBV,128.94,31.07,N/A
...,...,...,...,...
500,YUM,135.02,26.32,N/A
501,ZBH,128.70,32.18,N/A
502,ZBRA,623.44,41.13,N/A
503,ZION,64.32,6.22,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [80]:
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe.sort_values('Price-to-Earnings Ratio',inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True,inplace=True)
final_dataframe

<ipython-input-80-2a8a197cc026>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.sort_values('Price-to-Earnings Ratio',inplace=True)


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,747.87,3.41,N/A
1,HPQ,36.50,3.46,N/A
2,EBAY,68.09,3.71,N/A
3,NRG,39.81,3.9,N/A
4,MPC,66.62,4.63,N/A
5,BEN,35.15,4.87,N/A
6,DHI,113.27,5,N/A
7,TSN,86.65,5.28,N/A
8,HOLX,75.02,5.36,N/A
9,LYB,94.21,5.46,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [81]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [82]:
portfolio_input()

Enter the value of your portfolio: 1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [83]:
position_size=float(portfolio_size)/len(final_dataframe.index)
position_size

20000.0

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [84]:
rv_column=[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    "Price-to-Earnings Ratio",
    "PE Percentile",
    "Price-to-Book Ratio",
    "PB Percentile",
    "Price-to-Sales Ratio",
    "PS Percentile",
    "EV/EBITDA",
    "EV/EBITDA Percentile",
    "EV/GP",
    "EV/GP Percentile",
    "RV Score"
]

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [92]:
rv_dataframe = pd.DataFrame(columns=rv_column)

for symbol_string in symbol_strings:
    api_url = f"/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN[-1]}"
    data = requests.get(base_url+api_url).json()
    
    for symbol in symbol_string.split(","):
        try:
            ev_ebitda=float(data[symbol]['advanced-stats']['enterpriseValue'])/float(data[symbol]['advanced-stats']['EBITDA'])
        except Exception:
            ev_ebitda=np.NaN
            
        try:
            ev_gp=float(data[symbol]['advanced-stats']['enterpriseValue'])/float(data[symbol]['advanced-stats']['grossProfit'])
        except Exception:
            ev_gp=np.NaN
        
        rv_dataframe=rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                "N/A",
                data[symbol]['advanced-stats']['peRatio'],
                "N/A",
                data[symbol]['advanced-stats']['priceToBook'],
                "N/A",
                data[symbol]['advanced-stats']['priceToSales'],
                "N/A",
                ev_ebitda,
                "N/A",
                ev_gp,
                "N/A",
                "N/A"
                
                
            ],index=rv_column
            ),ignore_index=True
        )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,163.79,N/A,47.869484,N/A,9.87,N/A,8.1,N/A,32.129319,N/A,14.871422,N/A,N/A
1,AAL,18.29,N/A,-3.697636,N/A,-1.63,N/A,0.4967,N/A,-7.196187,N/A,1.468989,N/A,N/A
2,AAP,247.38,N/A,23.686086,N/A,4.82,N/A,1.4,N/A,14.046319,N/A,3.221870,N/A,N/A
3,AAPL,184.65,N/A,15.737321,N/A,23.67,N/A,4.18,N/A,12.889798,N/A,9.875837,N/A,N/A
4,ABBV,131.70,N/A,30.931073,N/A,16.78,N/A,4.15,N/A,10.594739,N/A,7.552436,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.35,N/A,25.321841,N/A,-5,N/A,6.1,N/A,22.703416,N/A,10.469088,N/A,N/A
501,ZBH,125.92,N/A,32.307728,N/A,2.07,N/A,3.4,N/A,18.039354,N/A,5.844282,N/A,N/A
502,ZBRA,623.88,N/A,40.670067,N/A,11.89,N/A,6.1,N/A,30.493711,N/A,12.928110,N/A,N/A
503,ZION,66.05,N/A,5.852689,N/A,1.37,N/A,2.93,N/A,4.840155,N/A,2.685997,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` metrichod to identify missing data:

In [86]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,186.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,294.410,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,300.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,23.520,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.630,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.580,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.520,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,3006.500,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,106.640,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` metrichod is useful here)
* Replace missing data with a new value (pandas' `fillna` metrichod is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [93]:
df_col = [ 
 'Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']

for column in df_col:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(),inplace=True)
    

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [94]:
rv_dataframe[rv_dataframe.isna().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [95]:
metrics={
     'Price-to-Earnings Ratio':'PE Percentile',
     'Price-to-Book Ratio':'PB Percentile',
     'Price-to-Sales Ratio':'PS Percentile',
     'EV/EBITDA':'EV/EBITDA Percentile',
     'EV/GP':'EV/GP Percentile'
}
metrics

{'Price-to-Earnings Ratio': 'PE Percentile',
 'Price-to-Book Ratio': 'PB Percentile',
 'Price-to-Sales Ratio': 'PS Percentile',
 'EV/EBITDA': 'EV/EBITDA Percentile',
 'EV/GP': 'EV/GP Percentile'}

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetricic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetricic mean, we will use the mean function from Python's built-in statistics module.

In [96]:
for row in rv_dataframe.index:
    rv_measure_list=[]
    for metric in metrics.keys():
        rv_dataframe.loc[row,metrics[metric]] = stats.percentileofscore(rv_dataframe[metric],rv_dataframe.loc[row,metric])/100
        rv_measure_list.append(rv_dataframe.loc[row,metrics[metric]])
        
    rv_dataframe.loc[row,'RV Score'] = mean(rv_measure_list)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,163.79,N/A,47.869484,0.809901,9.87,0.80396,8.1000,0.827723,32.129319,0.863366,14.871422,0.851485,0.831287
1,AAL,18.29,N/A,-3.697636,0.053465,-1.63,0.055446,0.4967,0.049505,-7.196187,0.023762,1.468989,0.063366,0.049109
2,AAP,247.38,N/A,23.686086,0.479208,4.82,0.605941,1.4000,0.2,14.046319,0.467327,3.221870,0.182178,0.386931
3,AAPL,184.65,N/A,15.737321,0.29703,23.67,0.942574,4.1800,0.611881,12.889798,0.423762,9.875837,0.665347,0.588119
4,ABBV,131.70,N/A,30.931073,0.633663,16.78,0.906931,4.1500,0.609901,10.594739,0.322772,7.552436,0.530693,0.600792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.35,N/A,25.321841,0.520792,-5.00,0.051485,6.1000,0.757426,22.703416,0.716832,10.469088,0.69901,0.549109
501,ZBH,125.92,N/A,32.307728,0.651485,2.07,0.29703,3.4000,0.524752,18.039354,0.60198,5.844282,0.386139,0.492277
502,ZBRA,623.88,N/A,40.670067,0.748515,11.89,0.843564,6.1000,0.757426,30.493711,0.835644,12.928110,0.784158,0.793861
503,ZION,66.05,N/A,5.852689,0.089109,1.37,0.150495,2.9300,0.460396,4.840155,0.067327,2.685997,0.134653,0.180396


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [97]:
rv_dataframe.sort_values('RV Score',ascending=False,inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True,inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,664.00,N/A,596.519053,0.99604,36.54,0.966337,23.35,0.984158,192.576135,0.99802,30.690446,0.984158,0.985743
1,DXCM,583.10,N/A,103.773937,0.964356,26.70,0.950495,24.03,0.986139,118.589258,0.99604,33.319584,0.990099,0.977426
2,NVDA,315.37,N/A,92.013687,0.950495,31.99,0.960396,32.20,0.99604,78.535958,0.980198,47.046095,0.99604,0.976634
3,PAYC,422.31,N/A,149.052575,0.974257,31.17,0.956436,25.96,0.988119,86.504586,0.988119,28.586444,0.976238,0.976634
4,FTNT,333.60,N/A,98.162590,0.962376,49.69,0.972277,17.81,0.958416,75.004239,0.976238,21.532078,0.940594,0.96198
5,IDXX,613.40,N/A,70.149851,0.924752,70.50,0.984158,16.57,0.952475,49.631316,0.956436,28.333869,0.972277,0.95802
6,INTU,682.45,N/A,92.717519,0.952475,19.31,0.922772,18.73,0.964356,67.050404,0.972277,21.924359,0.942574,0.950891
7,ISRG,349.34,N/A,76.398331,0.938614,10.80,0.830693,23.06,0.982178,56.581080,0.960396,31.037329,0.986139,0.939604
8,CDNS,185.09,N/A,75.111025,0.930693,19.79,0.924752,17.53,0.956436,62.750775,0.970297,18.745012,0.910891,0.938614
9,TYL,537.22,N/A,137.817606,0.968317,9.77,0.79802,15.05,0.940594,101.225121,0.992079,33.733350,0.992079,0.938218


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [98]:
portfolio_input()

Enter the value of your portfolio: 100000


In [110]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row,'Price'])
rv_dataframe

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,664.00,3,596.519053,0.99604,36.54,0.966337,23.35,0.984158,192.576135,0.99802,30.690446,0.984158,0.985743
1,DXCM,583.10,3,103.773937,0.964356,26.70,0.950495,24.03,0.986139,118.589258,0.99604,33.319584,0.990099,0.977426
2,NVDA,315.37,6,92.013687,0.950495,31.99,0.960396,32.20,0.99604,78.535958,0.980198,47.046095,0.99604,0.976634
3,PAYC,422.31,4,149.052575,0.974257,31.17,0.956436,25.96,0.988119,86.504586,0.988119,28.586444,0.976238,0.976634
4,FTNT,333.60,5,98.162590,0.962376,49.69,0.972277,17.81,0.958416,75.004239,0.976238,21.532078,0.940594,0.96198
5,IDXX,613.40,3,70.149851,0.924752,70.50,0.984158,16.57,0.952475,49.631316,0.956436,28.333869,0.972277,0.95802
6,INTU,682.45,2,92.717519,0.952475,19.31,0.922772,18.73,0.964356,67.050404,0.972277,21.924359,0.942574,0.950891
7,ISRG,349.34,5,76.398331,0.938614,10.80,0.830693,23.06,0.982178,56.581080,0.960396,31.037329,0.986139,0.939604
8,CDNS,185.09,10,75.111025,0.930693,19.79,0.924752,17.53,0.956436,62.750775,0.970297,18.745012,0.910891,0.938614
9,TYL,537.22,3,137.817606,0.968317,9.77,0.79802,15.05,0.940594,101.225121,0.992079,33.733350,0.992079,0.938218


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [111]:
writer = pd.ExcelWriter('value_strategy.xlsx',engine='xlsxwriter')

rv_dataframe.to_excel(writer, "Value Strategy",index = False)
background_color = '#0a0a23'
font_color = '#ffffff'

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [112]:
string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [113]:
column_formats={
     'A':['Ticker', string_template],
     'B':['Price', dollar_template],
     'C':['Number of Shares to Buy', integer_template],
     'D':['Price-to-Earnings Ratio', float_template],
     'E':['PE Percentile', percent_template],
     'F':['Price-to-Book Ratio', float_template],
     'G':['PB Percentile', percent_template],
     'H':['Price-to-Sales Ratio', float_template],
     'I':['PS Percentile', percent_template],
     'J':['EV/EBITDA', float_template],
     'K':['EV/EBITDA Percentile', percent_template],
     'L':['EV/GP', float_template],
     'M':['EV/GP Percentile', percent_template],
     'N':['RV Score', percent_template]
}

for colum in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f"{colum}:{colum}",25,column_formats[colum][1])
    writer.sheets['Value Strategy'].write(f"{colum}1",column_formats[colum][0],column_formats[colum][1])


## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [114]:
writer.save()